# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-08-02 to 2021-07-30
You have 435 stocks
['A', 'AAP', 'ABBV', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AME', 'AMGN', 'AMP', 'AMT', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CFG', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DOV', 'DOW', 'DPZ', 'DRE', 'DR

ticker,A,AAP,ABBV,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,...,WYNN,XEL,XLNX,XOM,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-07-26 00:00:00+00:00,150.25,214.09,117.79,325.00,119.52,318.98,620.80,163.90,58.44,206.27,...,104.32,68.31,135.44,58.48,124.08,123.45,159.65,545.31,51.48,200.82
2021-07-27 00:00:00+00:00,149.95,215.37,117.96,321.96,119.81,319.89,618.28,162.59,58.85,207.89,...,100.34,69.46,131.00,57.83,123.95,125.48,162.84,540.18,51.68,201.87
2021-07-28 00:00:00+00:00,151.46,212.25,118.55,323.80,120.52,316.31,620.92,164.04,58.64,206.88,...,102.16,68.70,138.54,58.22,122.93,122.61,162.56,545.74,52.36,203.27
2021-07-29 00:00:00+00:00,152.67,213.39,118.87,324.81,121.09,318.35,621.70,166.54,59.60,208.83,...,99.76,68.79,147.25,58.93,125.06,130.31,162.74,548.10,52.84,204.12
2021-07-30 00:00:00+00:00,153.23,212.06,116.30,327.14,120.98,317.68,621.63,167.42,59.72,209.63,...,98.33,68.25,149.84,57.57,125.85,131.39,163.42,552.48,52.15,202.70


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [4]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [5]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAPL'].tail()

,,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day
date,ticker,,,


In [6]:
all_factors.describe()

,overnight_sentiment_5_day_smoothed,annualzed_volatility_20_day,annualzed_volatility_120_day
count,4.432650e+05,4.432650e+05,4.432650e+05
mean,1.001859e-19,1.442678e-19,-3.807066e-19
std,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728074e+00,-1.728074e+00,-1.728074e+00
25%,-8.680186e-01,-8.680186e-01,-8.680186e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.680186e-01,8.680186e-01,8.680186e-01
max,1.728074e+00,1.728076e+00,1.728076e+00


In [7]:
all_factors.head()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2017-07-14 00:00:00+00:00 A                                 1.313973   
                          AAP                              -1.648439   
                          ABBV                             -1.019325   
                          ABMD                              1.600658   
                          ABT                              -0.629114   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2017-07-14 00:00:00+00:00 A                          0.668933   
                          AAP                        1.552879   
                          ABBV                      -0.087598   
                          ABMD                       0.581334   
                          ABT                        1.083034   

                                  annualzed_volatility_120_day  
date                      ticker                                
2017-07-14 00:00:00+00:00 A                           0.382247  
                          AAP                         1.544916  
                          ABBV                        0.366320  
                          ABMD                        1.313975  
                          ABT                         0.461882

In [8]:
all_factors.tail()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-07-30 00:00:00+00:00 YUM                               1.568804   
                          ZBH                               1.106923   
                          ZBRA                              0.517626   
                          ZION                              1.560841   
                          ZTS                               0.207050   

                                  annualzed_volatility_20_day  \
date                      ticker                                
2021-07-30 00:00:00+00:00 YUM                        1.345827   
                          ZBH                        0.079635   
                          ZBRA                      -0.780420   
                          ZION                       0.756530   
                          ZTS                       -1.154704   

                                  annualzed_volatility_120_day  
date                      ticker                                
2021-07-30 00:00:00+00:00 YUM                        -1.489169  
                          ZBH                        -0.286685  
                          ZBRA                        1.106923  
                          ZION                        1.497133  
                          ZTS                         0.278722